# View a slice of the Autzen point cloud

The data used in this notebook can be [found here](https://github.com/PDAL/data/tree/master/autzen) and has a BSD license as [described here](https://pdal.io/en/latest/copyright.html#overall-pdal-license-bsd).

In [ ]:
import pandas as pd
import tiledb
from pybabylonjs import Show as show

## Data sources

A slice of a point cloud can be viewed from three different sources specified by the `source` parameter:
* a TileDB Cloud array (`source = "cloud"`)
* a local TileDB array (`source = "local"`)
* a dictionary (`source = "dict"`)

In all cases a bounding box (`bbox`) with the minimum and maximum values of X, Y and Z is needed to slice the data from the array:

In [ ]:
bbox = {
    'X': [636800, 637200],
    'Y': [852800, 853100],
    'Z': [406.14, 615.26]
}

### Cloud array

To view point cloud data from a TileDB cloud array a `token` is needed: 
* [sign up for a TileDB account](https://cloud.tiledb.com/auth/signup)

When running this notebook locally:
* [create a token as described here](https://docs.tiledb.com/cloud/how-to/account/create-api-tokens)
* uncomment the below cell and add your token (`<token>`)
* run the below cells

When running this notebook on TileDB Cloud:
* the token will be automatically loaded
* remove the token from the list of parameters of `show.point_cloud`
* run the below cells

In [ ]:
#token = "<token>"

In [ ]:
show.point_cloud(source="cloud",
                 uri = "tiledb://TileDB-Inc/autzen_classified_tiledb",
                 token=token,
                 bbox = bbox,
                 point_size = 3,
                 rgb_max = 65535,
                 camera_up = 25,
                 camera_location = 2,
                 camera_zoom = [2,2,2],
                 point_type = 'fixed_world_size',
                 width=1000,
                 height=600)

## Optional: create and view a point cloud array from a LAZ file

In [ ]:
import pdal

In [ ]:
!wget -nc "https://github.com/PDAL/data/blob/master/autzen/autzen-classified.laz?raw=true" -O "autzen-classified.laz"

In [ ]:
pipeline = (
  pdal.Reader("autzen-classified.laz") |
  pdal.Filter.stats() |
  pdal.Writer.tiledb(array_name="autzen-classified",chunk_size=100000)
)

count = pipeline.execute()  

### Local array
The point cloud data from the newly created array can now be viewed with the below. Note that a larger slice is loaded than in the cells above.

In [ ]:
lidar_array = "autzen-classified"

In [ ]:
bbox2 = {
    'X': [636800, 637800],
    'Y': [851000, 853000],
    'Z': [406.14, 615.26]
}

In [ ]:
show.point_cloud(source="local",
                 uri=lidar_array,
                 bbox = bbox2,
                 width = 1000,
                 height = 800,
                 point_size = 3,
                 rgb_max = 65535,
                 camera_up = 25,
                 camera_location = 2,
                 camera_zoom = [2,2,2],
                 point_type = 'fixed_world_size')

### Data from a dictionary
Alternatively data can be loaded into a dictionary first and then displayed. 

Load the data directly into a dictionary from the local array:

In [ ]:
with tiledb.open(lidar_array) as arr:
    data = arr.query(attrs=["Red", "Green", "Blue"], dims=["X", "Y", "Z"])[
        bbox2["X"][0] : bbox2["X"][1],
        bbox2["Y"][0] : bbox2["Y"][1],
        bbox2["Z"][0] : bbox2["Z"][1],
    ]

Or first load the data into a pandas DataFrame when for example pre-processing of the data is needed:

In [ ]:
with tiledb.open(lidar_array) as arr:
    df = pd.DataFrame(arr[
        bbox2["X"][0] : bbox2["X"][1],
        bbox2["Y"][0] : bbox2["Y"][1],
        bbox2["Z"][0] : bbox2["Z"][1]])

In [ ]:
df = df.drop(['ReturnNumber', 'NumberOfReturns', 'ScanDirectionFlag', 'EdgeOfFlightLine', 'ScanAngleRank', 'UserData', 'PointSourceId', 'ScanChannel', 'ClassFlags'], axis=1)

In [ ]:
df

In [ ]:
data = {
    'X': df['X'],
    'Y': df['Y'],
    'Z': df['Z'],
    'Red': df['Red'],
    'Green': df['Green'],
    'Blue': df['Blue']
}

In [ ]:
show.point_cloud(source="dict",
                 data=data,
                 bbox = bbox2,
                 point_size = 3,
                 width = 1000,
                 height = 700,
                 rgb_max = 65535,
                 camera_up = 25,
                 camera_location = 2,
                 camera_zoom = [2,2,2])